<a href="https://colab.research.google.com/github/LuisDi98/Audio_a_Lesco/blob/colab/Audio_To_LESCO_models_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install dependencies
! pip install datasets
! pip install transformers
! pip install -U openai-whisper
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer
! pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 9.7 MB/s eta 0

# Get Dataset
CIEMPIESS_TEST

In [ ]:
#Load the dataset
from datasets import load_dataset, load_metric, Audio
import whisper
ds=load_dataset("ciempiess/ciempiess_test", split="test")

#Downsample to 16kHz
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
ds

Dataset({
    features: ['audio_id', 'audio', 'speaker_id', 'gender', 'duration', 'normalized_text'],
    num_rows: 3558
})

In [ ]:
ds[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac',
 'array': array([-0.00192261, -0.00234985, -0.00158691, ..., -0.00839233,
        -0.00900269, -0.00698853]),
 'sampling_rate': 16000}

In [ ]:
ds.num_rows

3558

#Whisper-large
* De momento no ocupamos los inputs, esta es una manera más sencilla.
* Queda pendiente probar wav2vec2-large-xlsr-53-spanish-ep5-944h: https://huggingface.co/carlosdanielhernandezmena/wav2vec2-large-xlsr-53-spanish-ep5-944h
* Comparar con el word error rate


In [ ]:
large_model = whisper.load_model("large")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = large_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|██████████████████████████████████████| 2.87G/2.87G [00:26<00:00, 118MiB/s]


Whisper result:   Pues ahí está la libertad de las posiciones, de a ver quién es pasivo, quién es activo, bla, bla, bla. Muchas cosas, ¿no? Pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#Whisper-medium

In [ ]:
medium_model = whisper.load_model("medium")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = medium_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 94.3MiB/s]


Whisper result:   Pues ahí está la libertad de las posiciones, de a ver quién es pasivo, quién es activo, bla bla bla. Muchas cosas, ¿no? Pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


In [ ]:
#PRUEBAS DE LUIS DIEGO
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="spanish", task="transcribe")


# load dummy dataset and read audio files
ds=load_dataset("ciempiess/ciempiess_test", split="test")
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features
print(input_features)
# generate token ids
predicted_ids = model.generate(input_features)
print(predicted_ids)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
print(transcription)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)


#Whisper-tiny

In [ ]:
tiny_model = whisper.load_model("tiny")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = tiny_model.transcribe(sample)
print("Whisper result: ",result["text"] )
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 50.7MiB/s]


Whisper result:   pues esta la libertad de las posiciones de haber quien es pasivo, que en el activo la muchas cosas, ¿no? pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#Whisper Small

In [ ]:
small_model = whisper.load_model("small")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = small_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 53.7MiB/s]


Whisper result:   pues ahí está la libertad de las posiciones de haber quién es pasivo, quién es activo, bla bla bla, muchas cosas ¿no? Pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#Haciendo unas pruebas con el voice detection y wav2vec


In [ ]:
!pip install pydub


In [ ]:
import ipywidgets as widgets
from IPython import display as disp
from IPython.display import display, Audio, clear_output
from google.colab import output
import base64
from pydub import AudioSegment
import io
import tempfile
import librosa

In [ ]:
def record_audio(seconds=3,
                 sample_rate=44100,
                 normalize_db=0.1):
    """Record audio from the browser in colab using javascript.
    Based on: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
    Args:
      seconds: Number of seconds to record.
      sample_rate: Resample recorded audio to this sample rate.
      normalize_db: Normalize the audio to this many decibels. Set to None to skip
        normalization step.
    Returns:
      An array of the recorded audio at sample_rate.
    """
    # Use Javascript to record audio.
    record_js_code = """
      const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
      const b2text = blob => new Promise(resolve => {
        const reader = new FileReader()
        reader.onloadend = e => resolve(e.srcElement.result)
        reader.readAsDataURL(blob)
      })
      var record = time => new Promise(async resolve => {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true })
        recorder = new MediaRecorder(stream)
        chunks = []
        recorder.ondataavailable = e => chunks.push(e.data)
        recorder.start()
        await sleep(time)
        recorder.onstop = async ()=>{
          blob = new Blob(chunks)
          text = await b2text(blob)
          resolve(text)
        }
        recorder.stop()
      })
      """
    print('Starting recording for {} seconds...'.format(seconds))
    display(disp.Javascript(record_js_code))
    audio_string = output.eval_js('record(%d)' % (seconds * 1000.0))
    print('Finished recording!')
    audio_bytes = base64.b64decode(audio_string.split(',')[1])
    return audio_bytes_to_np(audio_bytes,
                             sample_rate=sample_rate,
                             normalize_db=normalize_db)

def audio_bytes_to_np(wav_data,
                      sample_rate=44100,
                      normalize_db=0.1):
    """Convert audio file data (in bytes) into a numpy array.
    Saves to a tempfile and loads with librosa.
    Args:
      wav_data: A byte stream of audio data.
      sample_rate: Resample recorded audio to this sample rate.
      normalize_db: Normalize the audio to this many decibels. Set to None to skip
        normalization step.
    Returns:
      An array of the recorded audio at sample_rate.
    """
    # Parse and normalize the audio.
    audio = AudioSegment.from_file(io.BytesIO(wav_data))
    audio.remove_dc_offset()
    if normalize_db is not None:
        audio.normalize(headroom=normalize_db)
    # Save to tempfile and load with librosa.
    with tempfile.NamedTemporaryFile(suffix='.wav') as temp_wav_file:
        fname = temp_wav_file.name
        audio.export(fname, format='wav')
        audio_np, unused_sr = librosa.load(fname, sr=sample_rate)
    return audio_np

In [ ]:
#@title Record
#@markdown * Set recording time:

SAMPLE_RATE = 44100
record_seconds =   10#@param {type:"number", min:1, max:10, step:1}

def _record_audio(b):
  global audio
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  display(Audio(audio, rate=SAMPLE_RATE))

button = widgets.Button(description="Start recording...")
button.on_click(_record_audio)
display(button)

# Normalización del texto

Necesitamos normalizar las salidas de whisper, ya que la librería nos proporciona las etiquetas de los audios pero estos están normalizados, este es un paso previo para poder aplicar algunas métricas y que los resultados sean más acertados.

In [ ]:
import re
from unicodedata import normalize

def formatText(entryText):
    '''
    Quita las etiquetas xml para extraer el texto, quita caracteres,convierte a minuscula
    y devuelve lista de palabras sin acentos (Como que hace más de una cosa xd)
    '''
    text = re.sub(r'<[^<]+>', "",entryText)
    text = text.lower()
    characters = "#&¿?¡!'-[]()\/''=`,:~}{+|." #-> caracteres innecesarios que se quitan
    text = ''.join(x for x in text if x not in characters)
    return text

def deleteAccents(text):
    '''
    Esta funcion no es necesaria para Whisper
    Borrar acentos del texto, exceptuando ñ
    #Obtenido de https://ideone.com/YcXaQD (ideone.com)
    '''
    #NFD (Normalization Form Canonical Decomposition) expresion para eliminar diacríticos
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    #NFC (#Normalization Form Canonical Composition) volver a su forma compuesta
    text = normalize( 'NFC', text)
    return text

def deleteSpaces(entryText):
    return " ".join(entryText.split())

def normalizeText(entryText):
    newText = formatText(entryText)
    return deleteSpaces(newText)

def normalizeTextDiacritics(entryText):
    newText = formatText(entryText)
    return deleteSpaces(deleteAccents(newText))

In [ ]:
normalizeText("No sé, honestamente hice hace mu#cho este código, cuando estaba en RIT & XD! [algo aqui] +, PERO me parece ineficiente")

'no sé honestamente hice hace mucho este código cuando estaba en rit xd algo aqui pero me parece ineficiente'

#WORD ERROR RATE
Se tiene una referencia, en este caso el label del audio y se tiene el texto generado por whisper.

WER = (S+D+I) / N => WER = (S+D+I) / (S+D+C)
​

Dónde:
* S es el número de sustituciones,
* D es el número de eliminaciones,
* I es el número de inserciones,
* C es el número de aciertos,
* N es el número de palabras en la referencia, N = (S + D + C).

A veces tambien se usa el Word accuracy, Auto Speech Recognition.

Wacc = 1 - WER

In [ ]:
import jiwer

def getWER(ref, hyp):
  error = jiwer.wer(ref, hyp)
  return error

In [ ]:
# El modelo que quieran
# tiny_model, medium_model, large_model
result = small_model.transcribe(sample)

In [ ]:
print("WER para el primer simple:", getWER((result["text"]), ds[0]["normalized_text"]))
print((result["text"]),"\n", ds[0]["normalized_text"])

WER para el primer simple: 0.4782608695652174
 pues ahí está la libertad de las posiciones de haber quién es pasivo, quién es activo, bla bla bla, muchas cosas ¿no? Pero... 
 pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


In [ ]:
print("WER para el primer texto normalizado:", getWER(normalizeText(result["text"]), ds[0]["normalized_text"]))
print(normalizeText(result["text"]),"\n", ds[0]["normalized_text"])

WER para el primer texto normalizado: 0.30434782608695654
pues ahí está la libertad de las posiciones de haber quién es pasivo quién es activo bla bla bla muchas cosas no pero 
 pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


In [ ]:
print("WER para el primer normalizado sin diacriticos:", getWER(normalizeTextDiacritics(result["text"]), ds[0]["normalized_text"]))
print(normalizeTextDiacritics(result["text"]),"\n", ds[0]["normalized_text"])

WER para el primer normalizado sin diacriticos: 0.43478260869565216
pues ahi esta la libertad de las posiciones de haber quien es pasivo quien es activo bla bla bla muchas cosas no pero 
 pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#WER para todo el data set de Ciempies con Whisper tiny
* Tiempo aproximado con 1000 audios:
  * Tiny: 7,5 mins
  * Small: 16,7 mins
  * Medium: 32,8 mins
  * Large: 46,5 mins



In [ ]:
import numpy as np

def getWERMetric(model, dataset, datasetSample):
  WERList = []
  for i in range(datasetSample):
    sample = dataset[i]["audio"]["path"]
    result = model.transcribe(sample)
    textResult = normalizeText(result["text"])
    label = dataset[i]["normalized_text"]
    if (textResult!="" and label!=""):
      WERList.append(getWER(textResult, label))
  return np.mean(WERList)

In [ ]:
import time

In [ ]:
start = time.time()
value = getWERMetric(tiny_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.2917494913844536
455.01868629455566


In [ ]:
start = time.time()
value = getWERMetric(small_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.18433433443007663
1003.1085572242737


In [ ]:
start = time.time()
value = getWERMetric(medium_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.17623326276082082
1970.6431975364685


In [ ]:
start = time.time()
value = getWERMetric(large_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.19101982265768988
2790.7779171466827


# NEW Voice Activity Detector


In [6]:
#@title Install and Import Dependencies
# this assumes that you have a relevant version of PyTorch installed
!pip install -q torchaudio
import torch
torch.set_num_threads(1)
from IPython.display import Audio
from pprint import pprint
USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
  !pip install -q onnxruntime

In [7]:
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)
(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


In [8]:
def chunk_audio(audio):
  readAudio = read_audio(audio, sampling_rate=16000)
  # get speech timestamps from full audio file
  speech_timestamps = get_speech_timestamps(readAudio, model, sampling_rate=16000)
  return (speech_timestamps,readAudio)

def getAudioVAD(audio):
   audio_chunks = chunk_audio(audio)
   return collect_chunks(audio_chunks[0], audio_chunks[1])

# merge all speech chunks to one audio
# audio_chunks = chunk_audio('/content/Prueba.m4a')
# save_audio('new_audio_VAD.wav', collect_chunks(audio_chunks[0], audio_chunks[1]), sampling_rate=16000)
# Audio('new_audio_VAD.wav')
# In this example the original audio was 17 sec, the new one is 10 s and VAD only took a couple of seconds

#Wav2VEC2


In [ ]:
import soundfile as sf
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")

# Set the desired tokenization parameters
max_length = 100  # Maximum length of the input in number of tokens
padding="True" # Whether to pad the sequences to have the same length
padding_side = "right"  # Side to apply padding on ("right" or "left")
truncation="True" # Whether to truncate the sequences if they exceed the maximum length
stride = 0.2  # Stride length for splitting audio into overlapping windows

# Update the tokenizer's parameters
tokenizer.model_max_length = max_length
tokenizer.padding = padding
tokenizer.padding_side = padding_side
tokenizer.truncation = truncation
tokenizer.stride = stride

model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


In [ ]:
def transcribe_audio(audio):
    input_values = tokenizer(audio, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    return transcription


In [ ]:
from jiwer import wer


In [ ]:
def conseguirWER(model,dataset,datasetSample):
  total_wer = 0.0
  total_audio_files = datasetSample

  for i in range(datasetSample):
      audio_path = dataset[i]["audio"]["path"]
      ground_truth_transcription = dataset[i]["normalized_text"]


      audio_dict = dataset[i]["audio"]
      audio, sample_rate = sf.read(audio_dict["path"])
      # Transcribe audio using the model
      transcription = transcribe_audio(audio)

      # Calculate Word Error Rate (WER)
      wer_score = wer(ground_truth_transcription, transcription)
      total_wer += wer_score

      average_wer = total_wer / total_audio_files
      print("Average WER:", average_wer)
      return average_wer


In [ ]:
start = time.time()
value = conseguirWER(model,ds,2000)
end = time.time()
print(value)
print(end-start)

Average WER: 0.0002380952380952381
0.0002380952380952381
11.46746039390564


In [ ]:
ds[0]["audio"]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac'

In [ ]:
ds[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac',
 'array': array([-0.00192261, -0.00234985, -0.00158691, ..., -0.00839233,
        -0.00900269, -0.00698853]),
 'sampling_rate': 16000}

In [ ]:
audio_dict = ds[0]["audio"]

In [ ]:
audio_dict["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac'

In [ ]:
import soundfile as sf

In [ ]:
audio, sample_rate = sf.read(audio_dict["path"])

# Test with large audios
Estas pruebas son usando los diferentes modelos de whisper y sin Voice Activity Detector.
1. Primero se debe correr el modelo de whisper que se desea.
2. Luego la funcion para transcirbir el audio, esta a su vez da el tiempo que se tarda.

Pruebas para el modelo Small
* Audio 30 mins           = 4.81 mins
* Audio 1 hora            = 5.99 mins
* Audio 1 hora y 30 mins  = 5.62 mins
* Audio 2 horas           = 6.68 mins
* Audio 2 horas y 30 mins = 15.1 mins
* Audio 3 horas           = 7.41 mins


In [2]:
from google.colab import drive
import whisper
import time
drive.mount('/content/drive')

small_model = whisper.load_model("small")
#tiny_model = whisper.load_model("tiny")
#medium_model = whisper.load_model("medium")
#large_model = whisper.load_model("large")


Mounted at /content/drive


100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 77.8MiB/s]


In [5]:
def transcribeAudio(audio, model):
  start = time.time()
  result = model.transcribe(audio)
  end = time.time()
  return (end-start, result)

In [ ]:
smallTime = {"30":0, "60":0, "90":0, "120":0, "150":0, "180":0}
audio1= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/30Min.mp3',small_model)
smallTime["30"] = audio1[0]
audio2= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/60Min.mp3',small_model)
smallTime["60"] = audio2[0]
audio3= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3',small_model)
smallTime["90"] = audio3[0]
audio4= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3',small_model)
smallTime["120"] = audio4[0]
audio5= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3',small_model)
smallTime["150"] = audio5[0]
audio6= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3',small_model)
smallTime["180"] = audio6[0]

In [ ]:
smallTime
'''
{'30': 289.04940462112427,
 '60': 359.7411675453186,
 '90': 337.9016089439392,
 '120': 400.85388445854187,
 '150': 906.0902349948883,
 '180': 1094.6138291358948
}
'''

Pruebas para el modelo Medium
* Audio 30 mins           =  5.94 mins
* Audio 1 hora            =  9.72 mins
* Audio 1 hora y 30 mins  =  15.91 mins
* Audio 2 horas           =  23.92 mins
* Audio 2 horas y 30 mins =  21.98 mins
* Audio 3 horas           =  20.80 mins

In [ ]:
mediumTime = {"30":0, "60":0, "90":0, "120":0, "150":0, "180":0}
audio1= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/30Min.mp3', medium_model)
mediumTime["30"] = audio1[0]
audio2= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/60Min.mp3', medium_model)
mediumTime["60"] = audio2[0]
audio3= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3', medium_model)
mediumTime["90"] = audio3[0]
audio4= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3', medium_model)
mediumTime["120"] = audio4[0]
audio5= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3', medium_model)
mediumTime["150"] = audio5[0]
audio6= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3', medium_model)
mediumTime["180"] = audio6[0]
#1 h,38 mins

In [ ]:
mediumTime

{'30': 356.7253592014313,
 '60': 583.5891742706299,
 '90': 954.7192177772522,
 '120': 1435.6262798309326,
 '150': 1319.2298076152802,
 '180': 1248.4831037521362}

# Test with VAD + Dataset

Modelo Small

In [9]:
smallTimeVAD = {"30":0, "60":0, "90":0, "120":0, "150":0, "180":0}
audio1= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/30Min.mp3'), small_model)
smallTimeVAD["30"] = audio1[0]
audio2= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/60Min.mp3'), small_model)
smallTimeVAD["60"] = audio2[0]
audio3= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3'), small_model)
smallTimeVAD["90"] = audio3[0]
audio4= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3'), small_model)
smallTimeVAD["120"] = audio4[0]
audio5= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3'), small_model)
smallTimeVAD["150"] = audio5[0]
audio6= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3'), small_model)
smallTimeVAD["180"] = audio6[0]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %669 : int[] = prim::profile_ivalue(%667)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)


In [12]:
smallTimeVAD

{'30': 174.00128769874573,
 '60': 314.86626648902893,
 '90': 396.40985894203186,
 '120': 550.9504287242889,
 '150': 658.8372821807861,
 '180': 775.54731798172}

Modelo medium

In [ ]:
mediumTimeVAD = {"30":0, "60":0, "90":0, "120":0, "150":0, "180":0}
audio1= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/30Min.mp3'), medium_model)
mediumTimeVAD["30"] = audio1[0]
audio2= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/60Min.mp3'), medium_model)
mediumTimeVAD["60"] = audio2[0]
audio3= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3'), medium_model)
mediumTimeVAD["90"] = audio3[0]
audio4= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3'), medium_model)
mediumTimeVAD["120"] = audio4[0]
audio5= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3'), medium_model)
mediumTimeVAD["150"] = audio5[0]
audio6= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3'), medium_model)
mediumTimeVAD["180"] = audio6[0]

In [ ]:
mediumTimeVAD